# The Battle of Neighbourhoods | Jupyter Notebook

# Analyzing The Neighbourhoods of Edmonton
## Author: Kenny Van

## Introduction
The purpose of this project is to preform an analysis of Edmonton, the capital of Alberta. Edmonton is the second most populous city in the province with a touch under a million residents as of 2016 spread over 685.25 km$^2$. Being a decently sized city there are a wide range of venues with multiple universities, shopping centres and recreational areas. 

We will be aiming to create an analysis using the techniques learned in the specialization to generate information useful to those who currently live in Edmonton and people interested in moving there. We will also generate information on what amenities are where for developers looking to start a business in Edmonton.

This project aims to determine the following properties about Edmonton:
1. Which neighbourhood has the most schools?
2. Which neighbourhood has the most recreational centers?
3. Are there regions of the city where restaurants are more accessible?
4. Are there regions of the city where grocery stores are more accessible?
5. Is there an even distribution of furniture stores?

The first four properties we will be trying to learn more about Edmonton directly influence where someone may choose to live in the city. Properties 3 and 4 would be more relevant to any developers in the food industry as entering an area with an over-abundance of either type of venue would be very difficult. Finally the last property would look at the number of furniture stores in Edmonton to determine if it would be reasonable to open a second Ikea in the area.


## Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Generate the Dataframe

### Scraping Wikipedia

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T"
extracting_data = requests.get(url).text
soup = BeautifulSoup(extracting_data,'lxml')

### Generate Dataframe with PostalCode, Borough, Neighbourhood, Longitude and Latitude

In [3]:
table = soup.find('table', {'class': 'wikitable'})
fields = table.find_all('td')

In [4]:
postalcode = []
borough = []
neighbourhood = []
longitude = []
latitude = []

for i in range(0, len(fields), 5):
    postalcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
    latitude.append(fields[i+3].text.strip())
    longitude.append(fields[i+4].text.strip())
        
alberta_df = pd.DataFrame(data=[postalcode, borough, neighbourhood, latitude, longitude]).transpose()
alberta_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']
print(alberta_df.shape)
alberta_df.head()

(180, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


### Remove any Boroughs that are not Assigned

In [5]:
alberta_df['Borough'].replace('Not assigned', np.nan, inplace=True)
alberta_df.dropna(subset=['Borough'], inplace=True)

print(alberta_df.shape)
alberta_df.head()

(144, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


### Remove and Latitudes that are not Assigned

In [6]:
alberta_df['Latitude'].replace('Not assigned', np.nan, inplace=True)
alberta_df.dropna(subset=['Latitude'], inplace=True)

print(alberta_df.shape)
alberta_df.head()

(138, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


### Filter For PostalCodes that are in Edmonton

In [7]:
edmonton_df = alberta_df[alberta_df['Borough'].str.contains('Edmonton')]

edmonton_df_sorted = edmonton_df.reset_index(drop=True)
print(edmonton_df_sorted.shape)
edmonton_df_sorted.head()

(38, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
1,T6A,Edmonton,North Capilano,53.5483,-113.408
2,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608
3,T6B,Edmonton,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404
4,T5C,Edmonton,Central Londonderry,53.6129,-113.4572


### Make sure our Longitude and Latitude Values are Floats

In [8]:
edmonton_df_sorted['Longitude'] = pd.to_numeric(edmonton_df_sorted["Longitude"])
edmonton_df_sorted['Latitude'] = pd.to_numeric(edmonton_df_sorted["Latitude"])

In [9]:
edmonton_df_sorted.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
1,T6A,Edmonton,North Capilano,53.5483,-113.4080
2,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608
3,T6B,Edmonton,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404
4,T5C,Edmonton,Central Londonderry,53.6129,-113.4572


## Generate a Map of Edmonton

In [10]:
address = 'Edmonton, Alberta'
geolocator = Nominatim(user_agent="edmonton_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Edmonton Canada are 53.535411, -113.507996.


In [11]:
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighbourhood in zip(edmonton_df_sorted['Latitude'], edmonton_df_sorted['Longitude'],
                                   edmonton_df_sorted['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.9,
        parse_html=False).add_to(map_edmonton)
map_edmonton

## Explore the Neighbourhoods Using FourSquare

In [12]:
CLIENT_ID = '1V3MBOSJQ4SLTJ3NQ51S5S5XXMSOTQHDFK5ZJZW5LG35IBHF' # your Foursquare ID
CLIENT_SECRET = 'PZJXYDGGFAZB4BUCZIWK4DEMD45C03UL4NZ24M5CGM1Z04MF' # your Foursquare Secret
VERSION = '20210302' # Foursquare API version
LIMIT = 150

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1V3MBOSJQ4SLTJ3NQ51S5S5XXMSOTQHDFK5ZJZW5LG35IBHF
CLIENT_SECRET:PZJXYDGGFAZB4BUCZIWK4DEMD45C03UL4NZ24M5CGM1Z04MF


### Define a Function that Grabs Nearby Venues

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
              
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

### Get the Venues

Choosing the appropriate radius is difficult, with a large radius we'll get significant overlap and if the radius is too small we wont accurately scan the neighbourhood. The area of Edmonton is:

$A_{\rm edm} = 684 \rm km^2$

Assuming that the neighbourhoods are evenly distributed across the city, Edmonton is a perfect circle, we can make a rough split of the city into equal sized chunks to get a search radius.

$N_{\rm neigh} = 38 \\$
$r_{\rm search} = \sqrt{\frac{A_{\rm edm}}  {\pi N_{\rm neigh}}}\\$ 
$r_{\rm search} \sim 2.4 \rm km$

We use a 2.5km search radius

In [14]:
edmonton_data = edmonton_df_sorted
edmonton_venues = getNearbyVenues(names=edmonton_data['Neighbourhood'],
                                  latitudes=edmonton_data['Latitude'],
                                  longitudes=edmonton_data['Longitude'],
                                  radius=2500) # 2.5km radius

West Clareview, East Londonderry
North Capilano
East North Central, West Beverly
SE Capilano, West Southeast Industrial, East Bonnie Doon
Central Londonderry
Central Bonnie Doon
West Londonderry, East Calder
South Bonnie Doon, East University
North Central, Queen Mary Park, Blatchford
West University, Strathcona Place
NorthDowntown Fringe, East Downtown Fringe
Southgate, North Riverbend
North Downtown
Kaskitayo, Aspen Gardens
South Downtown, South Downtown Fringe (Alberta Provincial Government)
West Mill Woods
North Westmount, West Calder, East Mistatim
East Mill Woods
South Westmount, Groat Estate, East Northwest Industrial
Southwest Edmonton
Glenora, SW Downtown Fringe
South Industrial
North Jasper Place
East Southeast Industrial, South Clover Bar
Central Jasper Place, Buena Vista
Southgate, North Riverbend
West Northwest Industrial, Winterburn
North Clover Bar
West Jasper Place, West Edmonton Mall
The Meadows
Central Mistatim
The Palisades, West Castle Downs
Central Beverly
Heritage

In [15]:
print(edmonton_venues.shape)
edmonton_venues.head()

(2449, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"West Clareview, East Londonderry",53.5899,-113.4413,Costco,53.595827,-113.416389,Warehouse Store
1,"West Clareview, East Londonderry",53.5899,-113.4413,World Health,53.598540,-113.415115,Gym
2,"West Clareview, East Londonderry",53.5899,-113.4413,Tim Hortons,53.589316,-113.419070,Coffee Shop
3,"West Clareview, East Londonderry",53.5899,-113.4413,Portuguese-Canadian Bakery,53.570478,-113.423432,Bakery
4,"West Clareview, East Londonderry",53.5899,-113.4413,Cinema City,53.590931,-113.420582,Movie Theater


In [16]:
edmonton_venues.groupby('Neighbourhood').count().head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Central Beverly,25,25,25,25,25,25
Central Bonnie Doon,100,100,100,100,100,100
"Central Jasper Place, Buena Vista",53,53,53,53,53,53
Central Londonderry,58,58,58,58,58,58
Central Mistatim,37,37,37,37,37,37


In [17]:
print('There are {} uniques categories.'.format(len(edmonton_venues['Venue Category'].unique())))

There are 208 uniques categories.


## Analysis
1. Which neighbourhood has the most schools?
2. Which neighbourhood has the most recreational centers?
3. Are there regions of the city where restaurants are more accessible?
4. Are there regions of the city where grocery stores are more accessible?
5. Is there an even distribution of furniture stores?

In [18]:
# one hot encoding
edmonton_onehot = pd.get_dummies(edmonton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
edmonton_onehot['Neighbourhood'] = edmonton_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [edmonton_onehot.columns[-1]] + list(edmonton_onehot.columns[:-1])
edmonton_onehot = edmonton_onehot[fixed_columns]

edmonton_onehot.shape

(2449, 209)

#### Group the rows together and find the frequency of occurance

In [19]:
edmonton_grouped = edmonton_onehot.groupby('Neighbourhood').mean().reset_index()
edmonton_grouped.head()

,Neighbourhood,Airport Lounge,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Beach,Beer Store,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,College Gym,College Residence Hall,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Motel,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel Agency,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Central Beverly,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.080000,0.000000,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.080000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.00,0.040000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.080000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.04,0.0,0.0,0.04,0.00,0.0,0.000000,0.0,0.0,0.040000,0.00,0.0,0.000000,0.00,0.0,0.04,0.000000,0.000000,0.0,0.040000,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.0,0.04,0.0,0.04,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.040000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.04,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.00,0.040000,0.000000,0.080000,0.040000,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.080000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.04,0.0,0.00000

### Output the Names of all Unique Venues in Edmonton

In [20]:
venue_names = []
for col in edmonton_grouped.columns: 
    venue_names.append(col)
    print(col)
# print(venue_names)

Neighbourhood
Airport Lounge
American Restaurant
Art Gallery
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
BBQ Joint
Bakery
Bank
Bar
Baseball Stadium
Beach
Beer Store
Bookstore
Bowling Alley
Brazilian Restaurant
Breakfast Spot
Brewery
Bubble Tea Shop
Buffet
Burger Joint
Burrito Place
Bus Station
Butcher
Café
Cajun / Creole Restaurant
Casino
Cheese Shop
Chinese Restaurant
Chocolate Shop
Climbing Gym
Clothing Store
Coffee Shop
College Gym
College Residence Hall
College Stadium
Comedy Club
Comfort Food Restaurant
Comic Shop
Community Center
Concert Hall
Construction & Landscaping
Convenience Store
Cosmetics Shop
Cupcake Shop
Dance Studio
Deli / Bodega
Department Store
Dessert Shop
Dim Sum Restaurant
Diner
Discount Store
Dog Run
Doner Restaurant
Donut Shop
Eastern European Restaurant
Electronics Store
Escape Room
Event Space
Factory
Falafel Restaurant
Farm
Farmers Market
Fast Food Restaurant
Filipino Restaurant
Fish & Chips Shop
Flea Market
Flower Shop
Fondue Restaurant
Food & Dr

## Analysis 1. Schools in Edmonton
Unfortunately based on the list of venue categories we can pull from the FourSquare API, there is no "school" venue category. We can try to use other venue categories that would realistically be associated with schools to try and learn as much as we can. 

Possible venue categories that can give us information about schools:
- Playgrounds
- Library
- College Gym / College Residence Hall / College Stadium

#### Learning More about Playgrounds

In [21]:
pg_inds = np.where(edmonton_venues['Venue Category'] == 'Playground')

In [22]:
edmonton_venues.loc[pg_inds]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


It looks like the playground isn't a school playground and instead an indoor playground area. This won't help us learn anymore about where schools are in Edmonton

#### Learning more about Libraries

In [23]:
lib_inds = np.where(edmonton_venues['Venue Category'] == 'Library')

In [24]:
edmonton_venues.loc[lib_inds]

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1251,East Mill Woods,53.4681,-113.4339,Mill Woods Library (EPL),53.456246,-113.430113,Library


Our Foursquare query only pulls one library so unfortunately this is also not very helpful

#### Learning more about Colleges

In [25]:
col_inds = np.where(edmonton_venues['Venue Category'].str.contains("College"))

In [26]:
college_venues = edmonton_venues.loc[col_inds]
college_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
465,"South Bonnie Doon, East University",53.5087,-113.5078,Van Vliet Centre,53.524148,-113.527142,College Gym
472,"South Bonnie Doon, East University",53.5087,-113.5078,Butterdome (Universiade Pavilion),53.523377,-113.527707,College Gym
476,"South Bonnie Doon, East University",53.5087,-113.5078,Lister Centre - University of Alberta,53.522386,-113.530665,College Residence Hall
592,"West University, Strathcona Place",53.5248,-113.5334,Van Vliet Centre,53.524148,-113.527142,College Gym
601,"West University, Strathcona Place",53.5248,-113.5334,Butterdome (Universiade Pavilion),53.523377,-113.527707,College Gym


#### Getting rid of Duplicates

In [27]:
college_venues["lat_lon"] = college_venues["Venue Latitude"].astype(str) + college_venues["Venue Longitude"].astype(str)

In [28]:
college_venues_unique = college_venues.drop_duplicates(subset=['lat_lon'])
college_venues_unique = college_venues_unique.iloc[:, :-1]
college_venues_unique

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
465,"South Bonnie Doon, East University",53.5087,-113.5078,Van Vliet Centre,53.524148,-113.527142,College Gym
472,"South Bonnie Doon, East University",53.5087,-113.5078,Butterdome (Universiade Pavilion),53.523377,-113.527707,College Gym
476,"South Bonnie Doon, East University",53.5087,-113.5078,Lister Centre - University of Alberta,53.522386,-113.530665,College Residence Hall
835,"Southgate, North Riverbend",53.4839,-113.5227,Foote Field,53.503740,-113.531130,College Stadium


In [29]:
len(edmonton_venues.loc[col_inds]["Venue"].unique())

4

We're getting some overlap for our results, but it looks like there are 4 venues associated with colleges.

## Results 1.  Schools in Edmonton
Unfortunately, with the venues that are returned by our FourSquare query, it is difficult to draw any clear conclusions about schools in Edmonton and which neighbourhoods have more schools. To properly perform an analysis of schools we would have to draw data from another source beyond Foursquare.

## Analysis 2. Recreational Centers

Unlike schools, Recreation centers are a venue that appear in our FourSquare query so we can check where those appear in Edmonton

In [30]:
rec_inds = np.where(edmonton_venues['Venue Category'].str.contains("Recreation"))
rec_venues = edmonton_venues.loc[rec_inds]
rec_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2004,The Meadows,53.4768,-113.3662,The Meadows Community Recreation Center,53.458418,-113.371115,Recreation Center
2352,"Horse Hill, East Lake District",53.6026,-113.3837,Clareview Community Recreation Centre,53.602283,-113.402008,Recreation Center
2445,West Lake District,53.5966,-113.4882,Grand Trunk Fitness and Leisure Centre,53.594820,-113.522077,Recreation Center


It looks like there are 3 recreation centers found using our query, but there are associated locations that we can also look at:
- Climbing Gym
- College Gym
- Gym / Fitness Center

In [31]:
gym_inds = np.where(edmonton_venues['Venue Category'].str.contains("Gym"))
gym_venues = edmonton_venues.loc[gym_inds]
gym_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"West Clareview, East Londonderry",53.5899,-113.4413,World Health,53.598540,-113.415115,Gym
32,"West Clareview, East Londonderry",53.5899,-113.4413,Londonderry Fitness & Leisure Centre,53.607303,-113.444491,Gym / Fitness Center
106,"East North Central, West Beverly",53.5766,-113.4608,Commonwealth Community Recreation Centre,53.558127,-113.476976,Gym
159,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Bonnie Doon Leisure Centre,53.523523,-113.453423,Gym Pool
187,Central Londonderry,53.6129,-113.4572,Londonderry Fitness & Leisure Centre,53.607303,-113.444491,Gym / Fitness Center


#### Combining our gyms and rec centres together

In [32]:
rec_gym_venues = pd.concat([rec_venues, gym_venues])
rec_gym_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2004,The Meadows,53.4768,-113.3662,The Meadows Community Recreation Center,53.458418,-113.371115,Recreation Center
2352,"Horse Hill, East Lake District",53.6026,-113.3837,Clareview Community Recreation Centre,53.602283,-113.402008,Recreation Center
2445,West Lake District,53.5966,-113.4882,Grand Trunk Fitness and Leisure Centre,53.594820,-113.522077,Recreation Center
1,"West Clareview, East Londonderry",53.5899,-113.4413,World Health,53.598540,-113.415115,Gym
32,"West Clareview, East Londonderry",53.5899,-113.4413,Londonderry Fitness & Leisure Centre,53.607303,-113.444491,Gym / Fitness Center


#### Removing the duplicates

In [33]:
rec_gym_venues['lat_lon'] = rec_gym_venues["Venue Latitude"].astype(str) + rec_gym_venues["Venue Longitude"].astype(str)

In [34]:
rec_gym_venues_unique = rec_gym_venues.drop_duplicates(subset=['lat_lon'])
rec_gym_venues_unique = rec_gym_venues_unique.iloc[:, :-1]

print(rec_gym_venues_unique.shape)
rec_gym_venues_unique.head()

(29, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2004,The Meadows,53.4768,-113.3662,The Meadows Community Recreation Center,53.458418,-113.371115,Recreation Center
2352,"Horse Hill, East Lake District",53.6026,-113.3837,Clareview Community Recreation Centre,53.602283,-113.402008,Recreation Center
2445,West Lake District,53.5966,-113.4882,Grand Trunk Fitness and Leisure Centre,53.594820,-113.522077,Recreation Center
1,"West Clareview, East Londonderry",53.5899,-113.4413,World Health,53.598540,-113.415115,Gym
32,"West Clareview, East Londonderry",53.5899,-113.4413,Londonderry Fitness & Leisure Centre,53.607303,-113.444491,Gym / Fitness Center


Looks like there are 26 gyms/rec centers in Edmonton, which neighbhourhoods have the most?

In [35]:
neighs_with_gym = rec_gym_venues_unique.groupby("Neighbourhood").count().sort_values("Venue", ascending=False)
neighs_with_gym.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"South Bonnie Doon, East University",3,3,3,3,3,3
"The Palisades, West Castle Downs",3,3,3,3,3,3
"South Westmount, Groat Estate, East Northwest Industrial",3,3,3,3,3,3
"Central Jasper Place, Buena Vista",2,2,2,2,2,2
"West Clareview, East Londonderry",2,2,2,2,2,2


South Bonnie Doon, East University has the most gyms/rec centers at 3.

In [36]:
print("Number of Neighbourhoods with a Gym: " + str(len(neighs_with_gym)))

Number of Neighbourhoods with a Gym: 19


### Find the Neighbourhoods without Physical Recreation

In [37]:
no_gyms = list(set(edmonton_data["Neighbourhood"]) - set(rec_gym_venues_unique["Neighbourhood"]))
no_gyms.sort()
print("Number of Neighbourhoods without a Gym: " + str(len(no_gyms)))
no_gyms


Number of Neighbourhoods without a Gym: 18


['Central Beverly',
 'Central Londonderry',
 'East Castledowns',
 'East Southeast Industrial, South Clover Bar',
 'Ellerslie',
 'Glenora, SW Downtown Fringe',
 'Heritage Valley',
 'Kaskitayo, Aspen Gardens',
 'North Capilano',
 'North Central, Queen Mary Park, Blatchford',
 'North Clover Bar',
 'North Jasper Place',
 'NorthDowntown Fringe, East Downtown Fringe',
 'South Downtown, South Downtown Fringe (Alberta Provincial Government)',
 'West Jasper Place, West Edmonton Mall',
 'West Londonderry, East Calder',
 'West Northwest Industrial, Winterburn',
 'West University, Strathcona Place']

In [38]:
no_rec_inds = np.array([])
for neighbourhood_name in no_gyms:
    temp_inds = np.where(edmonton_data['Neighbourhood'].str.contains(neighbourhood_name))
    no_rec_inds = np.append(no_rec_inds, temp_inds)

no_rec_neighs = edmonton_data.loc[no_rec_inds]
print(no_rec_neighs.shape)
no_rec_neighs.head()

(17, 5)


/home/kenny/anaconda3/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
32.0,T5W,Edmonton,Central Beverly,53.5766,-113.4608
4.0,T5C,Edmonton,Central Londonderry,53.6129,-113.4572
34.0,T5X,Edmonton,East Castledowns,53.6072,-113.5183
23.0,T6P,Edmonton,"East Southeast Industrial, South Clover Bar",53.4996,-113.3678
35.0,T6X,Edmonton,Ellerslie,53.4154,-113.4917


### Create a Map of These Venues

In [65]:
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, venue in zip(rec_gym_venues_unique['Venue Latitude'], rec_gym_venues_unique['Venue Longitude'],
                           rec_gym_venues_unique['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.4,
        parse_html=False).add_to(map_edmonton)
for lat, lng, venue in zip(edmonton_df_sorted['Latitude'], edmonton_df_sorted['Longitude'],
                           edmonton_df_sorted['Neighbourhood']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='blue',
        fill=True,
        fill_color='87cefa',
        fill_opacity=0.1,
        parse_html=False).add_to(map_edmonton)
map_edmonton

## Results 2. Recreational Centers
There is a distinct lack of gyms/rec centers in the East central region and southwest Edmonton. The neighbourhood with the most rec centers is "South Bonnie Doon, East University". Unfortunately, due to the overlap of our search ranges, some of our neighbourhoods were incorrectly labelled as not having a gym/rec center nearby. Doing a quick visual check with the map we find that only 6 neighbourhoods are lacking lacking a gym/rec center, these are:
- North Clover Bar
- North Capilano
- SE Capilano, West Southeast Industrial, East Bonnie Doon
- East Southeast Industrial, South Clover Bar
- Ellerslie
- Heritage Valley


## Analysis 3. Restaurants
Unlike the previous two sections, finding all of the restaurants will be difficult as there isn't a unified category. Lets look at the columns again to determine what Venue types are of interest

In [40]:
for col in edmonton_grouped.columns: 
    venue_names.append(col)
    print(col)

Neighbourhood
Airport Lounge
American Restaurant
Art Gallery
Arts & Crafts Store
Asian Restaurant
Athletics & Sports
BBQ Joint
Bakery
Bank
Bar
Baseball Stadium
Beach
Beer Store
Bookstore
Bowling Alley
Brazilian Restaurant
Breakfast Spot
Brewery
Bubble Tea Shop
Buffet
Burger Joint
Burrito Place
Bus Station
Butcher
Café
Cajun / Creole Restaurant
Casino
Cheese Shop
Chinese Restaurant
Chocolate Shop
Climbing Gym
Clothing Store
Coffee Shop
College Gym
College Residence Hall
College Stadium
Comedy Club
Comfort Food Restaurant
Comic Shop
Community Center
Concert Hall
Construction & Landscaping
Convenience Store
Cosmetics Shop
Cupcake Shop
Dance Studio
Deli / Bodega
Department Store
Dessert Shop
Dim Sum Restaurant
Diner
Discount Store
Dog Run
Doner Restaurant
Donut Shop
Eastern European Restaurant
Electronics Store
Escape Room
Event Space
Factory
Falafel Restaurant
Farm
Farmers Market
Fast Food Restaurant
Filipino Restaurant
Fish & Chips Shop
Flea Market
Flower Shop
Fondue Restaurant
Food & Dr

Amongst our venue names, the following are of interest:
- America Restaurant
- Asian Restaurant
- BBQ Joint
- Bakery
- Bar
- Bistro
- Brazilian Restaurant
- Breakfast Spot
- Brewery
- Bubble Tea Shop
- Buffet
- Burger Joint
- Burrito Place
- Cafe
- Cajun / Creole Restaurant
- Chinese Restaurant
- Coffee Shop
- Comfort Food Restaurant
- Deli / Bodega
- Dessert Shop
- Dim sum Restaurant
- Diner 
- Donut Shop
- Eastern European Restaurant
- Falafel Restaurant
- Fast Food Restaurant
- Filipino Restaurant
- Fish & chips Shop
- Fondue Restaurant
- Food & Drink Shop
- Food Court
- Food Truck
- French Restaurant
- Fried Chicken Joint
- Gaming Cafe
- Gastropub
- German Restaurant
- Greek Restaurant
- Halal Restaurant
- Hot Dog Joint
- Hotpot Restaurant
- Ice Cream Shop
- Indian Restaurant
- Irish Pub
- Italian Restaurant
- Japanese Restaurant
- Korean Restaurant
- Latin American Restaurant
- Malay Restaurant
- Mediterranean Restaurant
- Mexican Restaurant
- Middle Eastern Restaurant
- New American Restaurant
- Noodle House
- Pizza Place
- Portugese Restaurant
- Poutine Place
- Pub 
- Ramen Restaurant
- Restaurant
- Salad Place
- Sandwich Place
- Scandanavian Restaurant
- Seafood Restaurant
- Shanghai Restaurant
- Soup Place
- Steakhouse
- Sushi Restaurant
- Tapas Restaurant
- Tea Room
- Thai Restaurant
- Turkish Restaurant
- Vegetarian / Vegan Restaurant
- Vietnamese Restaurant
- Wings Joint

This is a _SIGNIFICANT_ number of possible venues

They can be classified under a few types of places with similar names:
- "Restaurants"
- "Joints"
- "Places"
- Others:
    - Bakery
    - Bar
    - Bistro
    - Breakfast Spot
    - Brewery
    - Bubble Tea Shop
    - Buffet
    - Cafe
    - Coffee Shop
    - Deli / Bodega
    - Dessert Shop
    - Diner 
    - Donut Shop
    - Fish & chips Shop
    - Food & Drink Shop
    - Food Court
    - Food Truck
    - Gaming Cafe
    - Gastropub
    - Ice Cream Shop
    - Irish Pub
    - Noodle House
    - Pub 
    - Steakhouse
    - Tea Room


### Finding all "Restaurants"

In [41]:
rest_inds = np.where(edmonton_venues['Venue Category'].str.contains("Restaurant"))
rest_venues = edmonton_venues.loc[rest_inds]
print(rest_venues.shape)
rest_venues.head()

(627, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,"West Clareview, East Londonderry",53.5899,-113.4413,Tim Hortons,53.602583,-113.445060,Restaurant
7,"West Clareview, East Londonderry",53.5899,-113.4413,"Tony Roma's Ribs, Seafood, & Steaks",53.600237,-113.444122,American Restaurant
17,"West Clareview, East Londonderry",53.5899,-113.4413,Earls Restaurant & Bar,53.594738,-113.414936,American Restaurant
20,"West Clareview, East Londonderry",53.5899,-113.4413,New York Fries - Edmonton Outlet Collection,53.601539,-113.447054,Restaurant
21,"West Clareview, East Londonderry",53.5899,-113.4413,Thai Express,53.602442,-113.444694,Thai Restaurant


### Finding all "Joints"

In [42]:
joint_inds = np.where(edmonton_venues['Venue Category'].str.contains("Joint"))
joint_venues = edmonton_venues.loc[joint_inds]
print(joint_venues.shape)
joint_venues.head()

(63, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
42,"West Clareview, East Londonderry",53.5899,-113.4413,Red Robin Gourmet Burgers and Brews,53.597565,-113.412076,Burger Joint
60,North Capilano,53.5483,-113.4080,Five Guys,53.540604,-113.420051,Burger Joint
83,North Capilano,53.5483,-113.4080,Fat Frank's Dog House,53.540400,-113.420240,Hot Dog Joint
99,"East North Central, West Beverly",53.5766,-113.4608,Otto,53.564457,-113.487175,Hot Dog Joint
114,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Five Guys,53.540604,-113.420051,Burger Joint


### Finding all "Places"

In [43]:
place_inds = np.where(edmonton_venues['Venue Category'].str.contains("Place"))
place_venues = edmonton_venues.loc[place_inds]
print(place_venues.shape)
place_venues.head()

(175, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
14,"West Clareview, East Londonderry",53.5899,-113.4413,Subway,53.601607,-113.447729,Sandwich Place
23,"West Clareview, East Londonderry",53.5899,-113.4413,Subway,53.591978,-113.466618,Sandwich Place
24,"West Clareview, East Londonderry",53.5899,-113.4413,Subway,53.591367,-113.417902,Sandwich Place
39,"West Clareview, East Londonderry",53.5899,-113.4413,Boston Pizza,53.598468,-113.426147,Pizza Place
56,"West Clareview, East Londonderry",53.5899,-113.4413,Rexall Place concession #7,53.571680,-113.454905,Pizza Place


### Finding all "Other" locations

In [44]:
other = ["Bakery",
         "Bar",
         "Bistro",
         "Breakfast Spot",
         "Brewery",
         "Bubble Tea Shop",
         "Buffet",
         "Cafe",
         "Coffee Shop",
         "Deli / Bodega",
         "Dessert Shop",
         "Diner",
         "Donut Shop",
         "Fish & chips Shop",
         "Food & Drink Shop",
         "Food Court",
         "Food Truck",
         "Gaming Cafe",
         "Gastropub",
         "Ice Cream Shop",
         "Irish Pub",
         "Noodle House",
         "Pub",
         "Steakhouse",
         "Tea Room"]

other_inds = np.array([])
for venue_name in other:
    temp_inds = np.where(edmonton_venues['Venue Category'].str.contains(venue_name))
    other_inds = np.append(other_inds, temp_inds)

other_venues = edmonton_venues.loc[other_inds]
print(other_venues.shape)
other_venues.head()

(531, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3.0,"West Clareview, East Londonderry",53.5899,-113.4413,Portuguese-Canadian Bakery,53.570478,-113.423432,Bakery
53.0,"West Clareview, East Londonderry",53.5899,-113.4413,McGavin's Bread Basket,53.586979,-113.417514,Bakery
93.0,"East North Central, West Beverly",53.5766,-113.4608,Handy Bakery,53.570399,-113.474202,Bakery
94.0,"East North Central, West Beverly",53.5766,-113.4608,Popular Bakery,53.570329,-113.484485,Bakery
253.0,Central Bonnie Doon,53.5182,-113.4769,La Boule Patisserie + Bakery,53.516743,-113.491320,Bakery


### Combine Together the DataFrames

In [45]:
# all_rests = [rest_venues, joint_venues, place_venues, other_inds]
all_rests_df = pd.concat([rest_venues, joint_venues, place_venues, other_venues])
print(all_rests_df.shape)
all_rests_df.head()

(1396, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6.0,"West Clareview, East Londonderry",53.5899,-113.4413,Tim Hortons,53.602583,-113.445060,Restaurant
7.0,"West Clareview, East Londonderry",53.5899,-113.4413,"Tony Roma's Ribs, Seafood, & Steaks",53.600237,-113.444122,American Restaurant
17.0,"West Clareview, East Londonderry",53.5899,-113.4413,Earls Restaurant & Bar,53.594738,-113.414936,American Restaurant
20.0,"West Clareview, East Londonderry",53.5899,-113.4413,New York Fries - Edmonton Outlet Collection,53.601539,-113.447054,Restaurant
21.0,"West Clareview, East Londonderry",53.5899,-113.4413,Thai Express,53.602442,-113.444694,Thai Restaurant


### Let's Remove any Duplicates

In [46]:
all_rests_df['unique_id'] = all_rests_df['Venue'] + all_rests_df["Venue Latitude"].astype(str) + all_rests_df["Venue Longitude"].astype(str)
all_rests_df_unique = all_rests_df.drop_duplicates(subset=['unique_id'])
all_rests_df_unique = all_rests_df_unique.iloc[:, :-1]
print(all_rests_df_unique.shape)
all_rests_df_unique.head()

(705, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6.0,"West Clareview, East Londonderry",53.5899,-113.4413,Tim Hortons,53.602583,-113.445060,Restaurant
7.0,"West Clareview, East Londonderry",53.5899,-113.4413,"Tony Roma's Ribs, Seafood, & Steaks",53.600237,-113.444122,American Restaurant
17.0,"West Clareview, East Londonderry",53.5899,-113.4413,Earls Restaurant & Bar,53.594738,-113.414936,American Restaurant
20.0,"West Clareview, East Londonderry",53.5899,-113.4413,New York Fries - Edmonton Outlet Collection,53.601539,-113.447054,Restaurant
21.0,"West Clareview, East Londonderry",53.5899,-113.4413,Thai Express,53.602442,-113.444694,Thai Restaurant


### Determine Which Neighbourhood has the Most Restaurants

In [47]:
neighs_with_food = all_rests_df_unique.groupby("Neighbourhood").count().sort_values("Venue", ascending=False)
neighs_with_food.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
South Industrial,46,46,46,46,46,46
West Mill Woods,45,45,45,45,45,45
"North Central, Queen Mary Park, Blatchford",44,44,44,44,44,44
"West Northwest Industrial, Winterburn",42,42,42,42,42,42
"NorthDowntown Fringe, East Downtown Fringe",42,42,42,42,42,42


In [48]:
print("Number of Neighbourhoods with a Restaurant: " + str(len(neighs_with_food)))

Number of Neighbourhoods with a Restaurant: 35


In [49]:
no_food = list(set(edmonton_data["Neighbourhood"]) - set(all_rests_df_unique["Neighbourhood"]))
no_food.sort()
print("Number of Neighbourhoods without a Restaurant: " + str(len(no_food)))
no_food


Number of Neighbourhoods without a Restaurant: 2


['Central Beverly', 'Kaskitayo, Aspen Gardens']

In [50]:
no_food_inds = np.array([])
for neighbourhood_name in no_food:
    temp_inds = np.where(edmonton_data['Neighbourhood'].str.contains(neighbourhood_name))
    no_food_inds = np.append(no_food_inds, temp_inds)

no_food_neighs = edmonton_data.loc[no_food_inds]
print(no_food_neighs.shape)
no_food_neighs.head()

(2, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
32.0,T5W,Edmonton,Central Beverly,53.5766,-113.4608
13.0,T6J,Edmonton,"Kaskitayo, Aspen Gardens",53.4822,-113.5269


### Plotting a Map of the Locations

In [51]:
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, venue in zip(all_rests_df_unique['Venue Latitude'], all_rests_df_unique['Venue Longitude'],
                           all_rests_df_unique['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='grey',
        fill_opacity=0.5,
        parse_html=False).add_to(map_edmonton)
for lat, lng, venue in zip(edmonton_df_sorted['Latitude'], edmonton_df_sorted['Longitude'],
                           edmonton_df_sorted['Neighbourhood']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='blue',
        fill=True,
        fill_color='87cefa',
        fill_opacity=0.1,
        parse_html=False).add_to(map_edmonton)
map_edmonton

## Results 3. Restaurants
Using the FoursQuare API we can see that restaurants almost cover the entire city of Edmonton. Our quick analysis suggests that three neighbourhoods lack a restaurant, but again, similar to the analysis above for gyms/rec centers this is a false negative due to the overlap of our search radius. There do appear to be two gaps in the restaurant coverage however, one clear gap in the southwest region and another in the eastern-central region of the city.

## Analysis 4. Grocery Stores
Furniture stores are an explicit category from the FourSquare API, so we can simply search for the venues that match with this specific category. In addition to simply "Grocery store" being a reasonable place to buy groceries, there are a few other venues that would be reasonable:
- Farmers Market
- Fruit & Vegetable Store
- Grocery Store
- Market
- Supermarket
- Warehouse Store

So we will gather the venues that fit all of these categories

In [52]:
grocery_cats = ["Farmers Market",
                "Fruit & Vegetable Store",
                "Grocery Store",
                "Market",
                "Supermarket",
                "Warehouse Store"]

In [53]:
gro_inds = np.array([])
for venue_name in grocery_cats:
    temp_inds = np.where(edmonton_venues['Venue Category'].str.contains(venue_name))
    gro_inds = np.append(gro_inds, temp_inds)

gro_venues = edmonton_venues.loc[gro_inds]
print(gro_venues.shape)
gro_venues.head()

(111, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
235.0,Central Bonnie Doon,53.5182,-113.4769,Old Strathcona Farmers' Market,53.519648,-113.495736,Farmers Market
404.0,"South Bonnie Doon, East University",53.5087,-113.5078,Old Strathcona Farmers' Market,53.519648,-113.495736,Farmers Market
1015.0,"South Downtown, South Downtown Fringe (Alberta...",53.5350,-113.5010,Old Strathcona Farmers' Market,53.519648,-113.495736,Farmers Market
70.0,North Capilano,53.5483,-113.4080,Ian's No Frills,53.569229,-113.394330,Grocery Store
153.0,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Safeway Bonnie Doon,53.519140,-113.456692,Grocery Store


### Let's Remove any Duplicates

In [54]:
gro_venues['unique_id'] = gro_venues['Venue'] + gro_venues["Venue Latitude"].astype(str) + gro_venues["Venue Longitude"].astype(str)
unique_grocery = gro_venues.drop_duplicates(subset=['unique_id'])
unique_grocery = unique_grocery.iloc[:, :-1]
print(unique_grocery.shape)
unique_grocery.head()

(57, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
235.0,Central Bonnie Doon,53.5182,-113.4769,Old Strathcona Farmers' Market,53.519648,-113.495736,Farmers Market
70.0,North Capilano,53.5483,-113.4080,Ian's No Frills,53.569229,-113.394330,Grocery Store
153.0,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Safeway Bonnie Doon,53.519140,-113.456692,Grocery Store
183.0,Central Londonderry,53.6129,-113.4572,Chris's No Frills,53.615640,-113.490379,Grocery Store
198.0,Central Londonderry,53.6129,-113.4572,Save-On-Foods,53.621775,-113.488552,Grocery Store


### Determine Which Neighbourhood Has the Most Grocery Stores

In [55]:
neighs_with_gro = unique_grocery.groupby("Neighbourhood").count().sort_values("Venue", ascending=False)
neighs_with_gro.head()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
West Mill Woods,5,5,5,5,5,5
Central Bonnie Doon,4,4,4,4,4,4
"The Palisades, West Castle Downs",4,4,4,4,4,4
Heritage Valley,4,4,4,4,4,4
"South Westmount, Groat Estate, East Northwest Industrial",4,4,4,4,4,4


In [56]:
print("Number of Neighbourhoods with a Grocery Store: " + str(len(neighs_with_gro)))

Number of Neighbourhoods with a Grocery Store: 28


In [57]:
no_gro = list(set(edmonton_data["Neighbourhood"]) - set(unique_grocery["Neighbourhood"]))
no_gro.sort()
print("Number of Neighbourhoods without a Grocery Store: " + str(len(no_gro)))
no_gro

Number of Neighbourhoods without a Grocery Store: 9


['Central Beverly',
 'East North Central, West Beverly',
 'Ellerslie',
 'Glenora, SW Downtown Fringe',
 'Kaskitayo, Aspen Gardens',
 'North Clover Bar',
 'North Downtown',
 'South Downtown, South Downtown Fringe (Alberta Provincial Government)',
 'Southwest Edmonton']

In [58]:
no_gro_inds = np.array([])
for neighbourhood_name in no_gro:
    temp_inds = np.where(edmonton_data['Neighbourhood'].str.contains(neighbourhood_name))
    no_gro_inds = np.append(no_gro_inds, temp_inds)

no_gro_neighs = edmonton_data.loc[no_gro_inds]
print(no_gro_neighs.shape)
no_gro_neighs.head()

(8, 5)


/home/kenny/anaconda3/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
32.0,T5W,Edmonton,Central Beverly,53.5766,-113.4608
2.0,T5B,Edmonton,"East North Central, West Beverly",53.5766,-113.4608
35.0,T6X,Edmonton,Ellerslie,53.4154,-113.4917
20.0,T5N,Edmonton,"Glenora, SW Downtown Fringe",53.5495,-113.5453
13.0,T6J,Edmonton,"Kaskitayo, Aspen Gardens",53.4822,-113.5269


## Plotting a Map of the Locations

In [59]:
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, venue in zip(unique_grocery['Venue Latitude'], unique_grocery['Venue Longitude'],
                           unique_grocery['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.5,
        parse_html=False).add_to(map_edmonton)
for lat, lng, venue in zip(edmonton_df_sorted['Latitude'], edmonton_df_sorted['Longitude'],
                           edmonton_df_sorted['Neighbourhood']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='blue',
        fill=True,
        fill_color='87cefa',
        fill_opacity=0.1,
        parse_html=False).add_to(map_edmonton)
map_edmonton

## Results 4. Grocery Stores
Again, unfortunately due to overlap of our search regions, some of our neighbourhoods are incorrectly classified as having no grocery stores. It appears as though all neighbourhoods have a grocery store nearby. similar to what was seen with restaurants, the southwest and east-central regions both appear to be lacking grocery stores.

## Analysis 5. Furniture Stores
Furniture stores are an explicit category from the FourSquare API, so we can simply search for the venues that match with this specific category.

In [60]:
furn_inds = np.where(edmonton_venues['Venue Category'].str.contains("Furniture"))
furn_venues = edmonton_venues.loc[furn_inds]
print(furn_venues.shape)
furn_venues.head()

(18, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
55,"West Clareview, East Londonderry",53.5899,-113.4413,ReStore,53.581531,-113.467127,Furniture / Home Store
171,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Habitat For Humanity ReStore,53.512118,-113.434290,Furniture / Home Store
379,"West Londonderry, East Calder",53.5923,-113.5168,Bed Bath & Beyond,53.600933,-113.490116,Furniture / Home Store
788,"Southgate, North Riverbend",53.4839,-113.5227,Crate & Barrel,53.484908,-113.515571,Furniture / Home Store
942,"Kaskitayo, Aspen Gardens",53.4822,-113.5269,Crate & Barrel,53.484908,-113.515571,Furniture / Home Store


In [61]:
furn_venues['unique_id'] = furn_venues['Venue'] + furn_venues["Venue Latitude"].astype(str) + furn_venues["Venue Longitude"].astype(str)
furn_venues_unique = furn_venues.drop_duplicates(subset=['unique_id'])
furn_venues_unique = furn_venues_unique.iloc[:, :-1]
print(furn_venues_unique.shape)
furn_venues_unique

(12, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
55,"West Clareview, East Londonderry",53.5899,-113.4413,ReStore,53.581531,-113.467127,Furniture / Home Store
171,"SE Capilano, West Southeast Industrial, East B...",53.5322,-113.4404,Habitat For Humanity ReStore,53.512118,-113.434290,Furniture / Home Store
379,"West Londonderry, East Calder",53.5923,-113.5168,Bed Bath & Beyond,53.600933,-113.490116,Furniture / Home Store
788,"Southgate, North Riverbend",53.4839,-113.5227,Crate & Barrel,53.484908,-113.515571,Furniture / Home Store
1509,South Industrial,53.4580,-113.4826,Bed Bath & Beyond,53.450059,-113.481155,Furniture / Home Store
1525,South Industrial,53.4580,-113.4826,IKEA Edmonton,53.442990,-113.489499,Furniture / Home Store
1811,"West Northwest Industrial, Winterburn",53.5416,-113.6249,Lee Valley Tools,53.545881,-113.640578,Furniture / Home Store
1885,"West Northwest Industrial, Winterburn",53.5416,-113.6249,Bed Bath & Beyond,53.523482,-113.626081,Furniture / Home Store
2048,Central Mistatim,53.5800,-113.5873,Best Plumbing & Lighting,53.569857,-113.586953,Furniture / Home Store
2106,"The Palisades, West Castle Downs",53.6202,-113.5430,HomeSense,53.600694,-113.557519,Furniture / Home Store


In [62]:
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighbourhood in zip(furn_venues_unique['Venue Latitude'], furn_venues_unique['Venue Longitude'],
                                   furn_venues_unique['Venue']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.5,
        parse_html=False).add_to(map_edmonton)
map_edmonton

## Results 5. Furniture Stores
Based on the results from the FourSquare query, there are only a handful of furniture stores in Edmonton. Four in the North, two on the West and three in the South totalling to nine stores in total. Based on our results it can be argued that the East portion of the city or Downtown could benefit from a furniture store.